In [205]:
#import libraries
import pandas as pd
import numpy as np

# reorganize worldBank

In [188]:
#import csv
worldBank = pd.read_csv('worldBank.csv')

#drop redundant columns
worldBank = worldBank.drop(columns='Indicator Code')
worldBank = worldBank.drop(columns='Country Code')

worldBank.head()

,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,Unnamed: 60
0,Arab World,% of females ages 15-49 having comprehensive c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,% of males ages 15-49 having comprehensive cor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,"Adolescent fertility rate (births per 1,000 wo...",133.555013,134.159119,134.857912,134.504576,134.105211,133.569626,132.675635,131.665502,...,49.999851,49.887046,49.781207,49.672975,49.536047,49.383745,48.796558,48.196418,NaN,NaN
3,Arab World,Adults (ages 15+) and children (0-14 years) li...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,Adults (ages 15+) and children (ages 0-14) new...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
#wide to long
worldBankMelt = worldBank.melt(id_vars=['Country Name', 'Indicator Name'], var_name='Year', value_name='Value')
worldBankMelt.head()

,Country Name,Indicator Name,Year,Value
0,Arab World,% of females ages 15-49 having comprehensive c...,1960,NaN
1,Arab World,% of males ages 15-49 having comprehensive cor...,1960,NaN
2,Arab World,"Adolescent fertility rate (births per 1,000 wo...",1960,133.555013
3,Arab World,Adults (ages 15+) and children (0-14 years) li...,1960,NaN
4,Arab World,Adults (ages 15+) and children (ages 0-14) new...,1960,NaN


In [190]:
#multiindex pivot function
def multiindex_pivot(df, columns=None, values=None):
    #https://github.com/pandas-dev/pandas/issues/23955
    names = list(df.index.names)
    df = df.reset_index()
    list_index = df[names].values
    tuples_index = [tuple(i) for i in list_index] # hashable
    df = df.assign(tuples_index=tuples_index)
    df = df.pivot(index="tuples_index", columns=columns, values=values)
    tuples_index = df.index  # reduced
    index = pd.MultiIndex.from_tuples(tuples_index, names=names)
    df.index = index
    return df

#set index
worldBankIndex = worldBankMelt.set_index(['Country Name', 'Year'])

#multiindex pivot
worldBankIndex = multiindex_pivot(worldBankIndex, 'Indicator Name', 'Value')

wb = worldBankIndex.reset_index()

wb = wb[wb['Year'] != 'Unnamed: 60']

wb['Year'] = wb['Year'].astype(int)

wb.head()

Indicator Name,Country Name,Year,% of females ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions),% of males ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions),AIDS estimated deaths (UNAIDS estimates),ARI treatment (% of children under 5 taken to a health provider),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Adults (ages 15+) and children (0-14 years) living with HIV,Adults (ages 15+) and children (ages 0-14) newly infected with HIV,Adults (ages 15+) living with HIV,...,Urban population,Urban population (% of total),Urban population growth (annual %),Urban poverty headcount ratio at national poverty lines (% of urban population),"Use of Intermittent Preventive Treatment of malaria, 2+ doses of SP/Fansidar (% of pregnant women)",Use of any antimalarial drug (% of pregnant women),Use of insecticide-treated bed nets (% of under-5 population),Vitamin A supplementation coverage rate (% of children ages 6-59 months),Wanted fertility rate (births per woman),Women's share of population ages 15+ living with HIV (%)
0,Afghanistan,1960,NaN,NaN,NaN,NaN,145.321,NaN,NaN,NaN,...,739462.0,8.221,5.253807,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1961,NaN,NaN,NaN,NaN,145.321,NaN,NaN,NaN,...,779754.0,8.508,5.305559,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1962,NaN,NaN,NaN,NaN,145.321,NaN,NaN,NaN,...,822719.0,8.805,5.363622,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1963,NaN,NaN,NaN,NaN,145.321,NaN,NaN,NaN,...,868325.0,9.110,5.395136,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1964,NaN,NaN,NaN,NaN,145.321,NaN,NaN,NaN,...,917022.0,9.426,5.456539,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# build qualifier dictionary

In [191]:
#function to rename country names in worldCup to match worldBank country names
def changeCountryName(country):
    if country == 'Yugoslavia': #Bosnia and Herzegovina, Croatia, Macedonia, Montanegro, Serbia, Slovenia
        return 'Serbia'
    if country == 'North Korea':
        return 'Korea, Dem. People’s Rep.'
    if country == 'Ivory Coast':
        return "Cote d'Ivoire"
    if country == 'Republic of Ireland':
        return 'Ireland'
    if country == 'Egypt':
        return 'Egypt, Arab Rep.'
    if country == 'Wales': #UK
        return 'United Kingdom'
    if country == 'Czechoslovakia':
        return 'Czech Republic' #Czech Republic, Slovakia
    if country == 'German DR':
        return 'Germany'
    if country == 'Slovakia':
        return 'Slovak Republic'
    if country == 'Scotland': #UK
        return 'United Kingdom'
    if country == 'DR Congo':
        return 'Congo, Dem. Rep.'
    if country == 'Northern Ireland': #UK
        return 'United Kingdom'
    if country == 'South Korea':
        return 'Korea, Rep.'
    if country == 'Russia':
        return 'Russian Federation'
    if country == 'China PR':
        return 'China'
    if country == 'England': #UK
        return 'United Kingdom'
    if country == 'Iran':
        return 'Iran, Islamic Rep.'
    return country

In [192]:
#all results
footballResults = pd.read_csv('all_int_football_results.csv')

#world cup results
worldCup = footballResults[footballResults['tournament'] == 'FIFA World Cup']

#create new year column from date column
worldCup['Year'] = worldCup['date'].str.split('-').str[0].astype(int)

#change worldCup country names to match worldBank country names
worldCup['home_team'] = worldCup.apply(lambda row: changeCountryName(row['home_team']), axis=1)
worldCup['away_team'] = worldCup.apply(lambda row: changeCountryName(row['away_team']), axis=1)

#limit worldCup data to years with worldBank data
worldCup = worldCup[worldCup['Year'] > 1958]

worldCup.head()

/Users/greggior/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/greggior/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/greggior/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Year
5109,1962-05-30,Argentina,Bulgaria,1,0,FIFA World Cup,Rancagua,Chile,True,1962
5110,1962-05-30,Brazil,Mexico,2,0,FIFA World Cup,Viña del Mar,Chile,True,1962
5111,1962-05-30,Chile,Switzerland,3,1,FIFA World Cup,Santiago,Chile,False,1962
5112,1962-05-30,Colombia,Uruguay,1,2,FIFA World Cup,Arica,Chile,True,1962
5113,1962-05-31,Czech Republic,Spain,1,0,FIFA World Cup,Viña del Mar,Chile,True,1962


In [193]:
#worldCup years
worldCupYears = worldCup['Year'].unique()
worldCupYears

array([1962, 1966, 1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002,
       2006, 2010, 2014, 2018])

In [194]:
#unique countries in worldCup
allHome = list(worldCup['home_team'].unique())
allAway = list(worldCup['away_team'].unique())
allTeams = allHome + allAway
allQualTeams = set(allTeams)

print(len(allQualTeams))

#unique countries in worldBank
worldBankCountries = list(wb['Country Name'].unique())
len(worldBankCountries)

#check for countries in worldCup not in worldBank
for country in allQualTeams:
    if country not in worldBankCountries:
        print(country)

73


In [195]:
#dictionary to hold {year:qualifying teams for that year}
qualDict = {}

for year in worldCupYears:
    home = list(worldCup[worldCup['Year']==year]['home_team'].unique())
    away = list(worldCup[worldCup['Year']==year]['away_team'].unique())
    teams = set(home + away)
    qualDict[year] = teams

# add boolean qualifier to worldBank

In [196]:
#function to check if country qualified for worldCup
def qualified(country, year):
    if year in qualDict:
        if country in qualDict[year]:
            return 1
        else:
            return 0
    else:
        return -1
    
wb['qualified'] = wb.apply(lambda row: qualified(row['Country Name'], row['Year']), axis=1)

#remove non worldCup years
wb = wb[wb['qualified'] != -1]
wb.head()

Indicator Name,Country Name,Year,% of females ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions),% of males ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions),AIDS estimated deaths (UNAIDS estimates),ARI treatment (% of children under 5 taken to a health provider),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Adults (ages 15+) and children (0-14 years) living with HIV,Adults (ages 15+) and children (ages 0-14) newly infected with HIV,Adults (ages 15+) living with HIV,...,Urban population (% of total),Urban population growth (annual %),Urban poverty headcount ratio at national poverty lines (% of urban population),"Use of Intermittent Preventive Treatment of malaria, 2+ doses of SP/Fansidar (% of pregnant women)",Use of any antimalarial drug (% of pregnant women),Use of insecticide-treated bed nets (% of under-5 population),Vitamin A supplementation coverage rate (% of children ages 6-59 months),Wanted fertility rate (births per woman),Women's share of population ages 15+ living with HIV (%),qualified
2,Afghanistan,1962,NaN,NaN,NaN,NaN,145.3210,NaN,NaN,NaN,...,8.805,5.363622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,Afghanistan,1966,NaN,NaN,NaN,NaN,145.3210,NaN,NaN,NaN,...,10.085,5.504228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
10,Afghanistan,1970,NaN,NaN,NaN,NaN,145.3210,NaN,NaN,NaN,...,11.529,5.797084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
14,Afghanistan,1974,NaN,NaN,NaN,NaN,147.1090,NaN,NaN,NaN,...,13.149,5.678049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
18,Afghanistan,1978,NaN,NaN,NaN,NaN,150.8782,NaN,NaN,NaN,...,14.958,4.463226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


# use wb to classify qualifiers vs. non-qualifiers
- select attributes of interest
- fill missing values
- normalize values

# goals!!!

In [241]:
wbQualifiers = wb[wb['qualified'] == 1]

wbQualifiers.head()

Indicator Name,Country Name,Year,% of females ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions),% of males ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions),AIDS estimated deaths (UNAIDS estimates),ARI treatment (% of children under 5 taken to a health provider),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Adults (ages 15+) and children (0-14 years) living with HIV,Adults (ages 15+) and children (ages 0-14) newly infected with HIV,Adults (ages 15+) living with HIV,...,Urban population (% of total),Urban population growth (annual %),Urban poverty headcount ratio at national poverty lines (% of urban population),"Use of Intermittent Preventive Treatment of malaria, 2+ doses of SP/Fansidar (% of pregnant women)",Use of any antimalarial drug (% of pregnant women),Use of insecticide-treated bed nets (% of under-5 population),Vitamin A supplementation coverage rate (% of children ages 6-59 months),Wanted fertility rate (births per woman),Women's share of population ages 15+ living with HIV (%),qualified
136,Algeria,1982,NaN,NaN,NaN,NaN,51.5940,NaN,NaN,NaN,...,45.303,5.088261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
140,Algeria,1986,NaN,NaN,NaN,NaN,36.7612,NaN,NaN,NaN,...,48.860,4.791444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
164,Algeria,2010,NaN,NaN,200.0,NaN,10.7334,6900.0,1000.0,6800.0,...,67.526,2.824082,NaN,NaN,NaN,NaN,NaN,NaN,44.225103,1
168,Algeria,2014,NaN,NaN,200.0,NaN,10.7914,8600.0,500.0,8400.0,...,70.129,2.826978,NaN,NaN,NaN,NaN,NaN,NaN,45.515507,1
331,Angola,2006,NaN,NaN,11000.0,NaN,191.9584,200000.0,27000.0,190000.0,...,36.942,5.563297,NaN,NaN,NaN,NaN,80.0,NaN,59.031023,1


In [242]:
worldCupHomeGoals = worldCup.groupby(['Year', 'home_team'])['home_score'].sum().to_frame().reset_index()
worldCupHomeGoals.head()

,Year,home_team,home_score
0,1962,Argentina,2
1,1962,Brazil,10
2,1962,Bulgaria,1
3,1962,Chile,10
4,1962,Colombia,5


In [243]:
worldCupAwayGoals = worldCup.groupby(['Year', 'away_team'])['away_score'].sum().to_frame().reset_index()
worldCupAwayGoals.head()

,Year,away_team,away_score
0,1962,Brazil,4
1,1962,Bulgaria,0
2,1962,Czech Republic,1
3,1962,Germany,2
4,1962,Hungary,8


In [244]:
worldCupGoals = worldCupHomeGoals.merge(worldCupAwayGoals, how='outer', left_on=['home_team', 'Year'], right_on=['away_team','Year'])
print(worldCupGoals.isna().sum())
worldCupGoals.head()

Year           0
home_team     29
home_score    29
away_team     29
away_score    29
dtype: int64


,Year,home_team,home_score,away_team,away_score
0,1962,Argentina,2.0,NaN,NaN
1,1962,Brazil,10.0,Brazil,4.0
2,1962,Bulgaria,1.0,Bulgaria,0.0
3,1962,Chile,10.0,NaN,NaN
4,1962,Colombia,5.0,NaN,NaN


In [245]:
def getTeam(home, away):
    if len(str(away)) > len(str(home)):
        return away
    else:
        return home

worldCupGoals['team'] = worldCupGoals.apply(lambda row: getTeam(row['home_team'], row['away_team']), axis=1)
print(worldCupGoals.isna().sum())

worldCupGoals = worldCupGoals.fillna(0)
worldCupGoals['totalGoals'] = worldCupGoals['home_score'] + worldCupGoals['away_score']
worldCupGoals = worldCupGoals.drop(columns=['home_team', 'away_team'])

worldCupGoals = worldCupGoals.rename(columns={'home_score':'totalHomeGoals', 'away_score':'totalAwayGoals'})

worldCupGoals.head()

Year           0
home_team     29
home_score    29
away_team     29
away_score    29
team           0
dtype: int64


,Year,totalHomeGoals,totalAwayGoals,team,totalGoals
0,1962,2.0,0.0,Argentina,2.0
1,1962,10.0,4.0,Brazil,14.0
2,1962,1.0,0.0,Bulgaria,1.0
3,1962,10.0,0.0,Chile,10.0
4,1962,5.0,0.0,Colombia,5.0


In [246]:
wbQ = wbQualifiers.merge(worldCupGoals, left_on=['Country Name', 'Year'], right_on=['team', 'Year'])

In [247]:
wbQ.head()

,Country Name,Year,% of females ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions),% of males ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions),AIDS estimated deaths (UNAIDS estimates),ARI treatment (% of children under 5 taken to a health provider),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Adults (ages 15+) and children (0-14 years) living with HIV,Adults (ages 15+) and children (ages 0-14) newly infected with HIV,Adults (ages 15+) living with HIV,...,Use of any antimalarial drug (% of pregnant women),Use of insecticide-treated bed nets (% of under-5 population),Vitamin A supplementation coverage rate (% of children ages 6-59 months),Wanted fertility rate (births per woman),Women's share of population ages 15+ living with HIV (%),qualified,totalHomeGoals,totalAwayGoals,team,totalGoals
0,Algeria,1982,NaN,NaN,NaN,NaN,51.5940,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,3.0,2.0,Algeria,5.0
1,Algeria,1986,NaN,NaN,NaN,NaN,36.7612,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,1.0,0.0,Algeria,1.0
2,Algeria,2010,NaN,NaN,200.0,NaN,10.7334,6900.0,1000.0,6800.0,...,NaN,NaN,NaN,NaN,44.225103,1,0.0,0.0,Algeria,0.0
3,Algeria,2014,NaN,NaN,200.0,NaN,10.7914,8600.0,500.0,8400.0,...,NaN,NaN,NaN,NaN,45.515507,1,1.0,6.0,Algeria,7.0
4,Angola,2006,NaN,NaN,11000.0,NaN,191.9584,200000.0,27000.0,190000.0,...,NaN,NaN,80.0,NaN,59.031023,1,0.0,1.0,Angola,1.0


# use wbQ goals for regression 
- select attributes of interest
- fill missing values
- normalize values